<h1> Notebook multiple compute units </h1>

In [1]:
DIR_PATH = "default/"
XCLBIN_PATH_DEFAULT = "/mnt/scratch/ldierick/octoray/fwi/bitstreams/u280_xclbin/500_500_HBM/FullW.xclbin"
XCLBIN_PATH_MULTCU = "/mnt/scratch/ldierick/octoray/fwi/bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin"

#DEVICE_NAME_DEFAULT="xilinx_u200_x"

In [2]:
def run_on_worker(grid_data,kernel,index):
    from dask import delayed, compute
    import time
    s = time.time()
    
    def execute_function(grid_data,kernel,path,index,config):
        import numpy as np
        import json
        import os, sys
        import time

        start_time = time.time()
        from pynq import Overlay, allocate, Device, lib


        from FWI.grid2D import grid2D
        from FWI.sources import Sources
        from FWI.receivers import Receivers
        from FWI.frequenciesGroup import frequenciesGroup
        from FWI.finiteDifferenceForwardModel import FiniteDifferenceForwardModel
        from FWI.conjugateGradientInversion import ConjugateGradientInversion

        devices = Device.devices
        ol = Overlay(path, download=True, device=devices[0])

        resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
        gridsize = len(grid_data)      

        print(resolution)
        print(gridsize)

        A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel[0]["dotprod_"+index][0]))
        B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel[0]["dotprod_"+index][1]))
        C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel[0]["dotprod_"+index][2]))

        D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel[1]["update_"+index][0]))
        E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel[1]["update_"+index][1]))
        F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel[1]["update_"+index][2]))

        # set up the kernel IP's
        dotprod = ol.dotprod_1
        update = ol.update_1

        print(f" import time {time.time() - start_time}")

        config["tolerance"] = 9.99*10**-7

        grid = grid2D([config["reservoirTopLeft"]["x"], config["reservoirTopLeft"]["z"]],
                      [config["reservoirBottomRight"]["x"], config["reservoirBottomRight"]["z"]],
                      [config["ngrid"]["x"], config["ngrid"]["z"]])
        source = Sources([config["sourcesTopLeft"]["x"], config["sourcesTopLeft"]["z"]],
                         [config["sourcesBottomRight"]["x"], config["sourcesBottomRight"]["z"]],
                         config["nSources"])
        receiver = Receivers([config["receiversTopLeft"]["x"], config["receiversTopLeft"]["z"]],
                             [config["receiversBottomRight"]["x"], config["receiversBottomRight"]["z"]],
                             config["nReceivers"])
        freq = frequenciesGroup(config["Freq"], config["c_0"])

        dot = []
        upd = []
        func_time = []
        rec_time = []
        chi = []

        print(f" len data: {len(grid_data)}")

        model = FiniteDifferenceForwardModel(grid, source, receiver, freq, None, True, gridsize,resolution, dotprod, update, A,B,C)

        inverse = ConjugateGradientInversion(None, model, config,D,E,F)
        config["max"] = 1000

        # Pre_processing
        referencePressureData = model.calculatePressureField(grid_data)

        rec_time_s = time.time()
        chi.extend(inverse.reconstruct(referencePressureData, config).data)
        rec_time.append(time.time() - rec_time_s)

        total_time = time.time() - start_time

        dot.append(model.dot_time)
        upd.append(inverse.updtime)

        A.freebuffer()
        B.freebuffer()
        C.freebuffer()
        D.freebuffer()
        E.freebuffer()
        F.freebuffer()
        
        del A
        del B
        del C
        del D
        del E
        del F
        ol.free()
        del ol
        

        import math
    #     rec_time.append(sum(rec_time))
    #     dot.append(sum(dot))
    #     upd.append(sum(upd))
    #     func_time.append(sum(func_time))
        dict_t = {
        "index": index,
        "time": {"total": total_time},
        }
    #     queue.put(dict_t)
        return dict_t
                

    #First check if this worker needs to handle multiple compute units.
    if kernel["no_instances"]==1:
        return execute_function(grid_data[0],kernel["functions"][0],kernel["path_to_kernel"],"1",kernel["config"])
    else:
        p = []
        for cu in range(kernel["no_instances"]):
            p.append(delayed(execute_function)(grid_data[cu],kernel["functions"][cu],kernel["path_to_kernel"],str(cu+1),kernel["config"]))
#         p1 = execute_function(grid_data[0],kernel["functions"][0],kernel["path_to_kernel"],"1",kernel["config"])
#         p2 = execute_function(grid_data[1],kernel["functions"][1],kernel["path_to_kernel"],"2",kernel["config"])
        p_res = compute(*p)
        t = time.time()-s
        return f"p:{p_res} t:{t}"
    
    
        


In [3]:
# first load in the data
import json
import copy
from Octoray import Octoray

octoray = Octoray(ssh_cluster=True, scheduler="10.1.212.129",scheduler_port=8786, hosts=["10.1.212.126"])

data = []
with open(DIR_PATH+"input/"+"10x10_100"+".txt") as f:
    for l in f:
        data.append(float(l))
    
# data.extend(data)

config = None
with open(DIR_PATH+"input/GenericInput.json") as f:
    config = json.load(f)


#set specific configurations for different number of compute units
single_cu_config = config
double_cu_config = copy.deepcopy(config)
single_cu_config["ngrid"]["x"]=50
double_cu_config["ngrid"]["x"]=25

# Then configure the kernels by specifying the path to the bitstream, number of compute units, batchsize per compute unit and the functions names and variables with their respective memory banks.
single_cu = octoray.create_kernel(XCLBIN_PATH_DEFAULT,1,500,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM3","HBM4","HBM5"]}]],single_cu_config)

double_cu = octoray.create_kernel(XCLBIN_PATH_MULTCU,2,250,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM6","HBM7","HBM8"]}],
                                                [{"dotprod_2":["HBM3","HBM4","HBM5"]},{"update_2":["HBM9","HBM10","HBM11"]}]],double_cu_config)

# Finally, add the kernels you want to execute and split the dataset based on the kernel configuration and the amount of workers available.

data_split, kernels_split = octoray.setup_cluster(data,single_cu)

results = octoray.execute_function(run_on_worker,data_split,kernels_split)

print(results)


Initializing OctoRay with client ip: 10.1.212.129
[ scheduler 10.1.212.129:8786 ] : /opt/tools/external/anaconda/envs/pynq-dask/bin/python -m distributed.cli.dask_scheduler --port 8786
[ worker 10.1.212.126 ] : /opt/tools/external/anaconda/envs/pynq-dask/bin/python -m distributed.cli.dask_worker 10.1.212.129:8786 --preload pynqimport.py --no-nanny -- memory-limit 0 --nthreads 1 --host 10.1.212.126


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/tools/external/anaconda/envs/pynq-dask/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/tools/external/anaconda/envs/pynq-dask/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ldierick/.local/lib/python3.7/site-packages/distributed/deploy/old_ssh.py", line 197, in async_ssh
    channel.send(b"\x03")  # Ctrl-C
  File "/opt/tools/external/anaconda/envs/pynq-dask/lib/python3.7/site-packages/paramiko/channel.py", line 801, in send
    return self._send(s, m)
  File "/opt/tools/external/anaconda/envs/pynq-dask/lib/python3.7/site-packages/paramiko/channel.py", line 1198, in _send
    raise socket.error("Socket is closed")
OSError: Socket is closed



Waiting until workers are set up on remote machines...


TimeoutError: Timed out after 15 seconds... exiting

In [ ]:
import time
q =time.time()
results = octoray.execute_function(run_on_worker,data_split,kernels_split)
print(f"TIME: {time.time()-q}")
print(results)


In [9]:
await octoray.close_workers()

In [6]:
import asyncssh

async with asyncssh.connect("10.1.212.129",22) as conn:
    res = await conn.run("ls",check=True)
    print(res.stdout,end='')

FWI
MySSHCluster.py
Octoray
Octoray.py
__pycache__
dask-worker-space
global.lock
multCU.ipynb
octoray
purge.lock
pynqimport.py


In [ ]:
import numpy as np

chi1 = [0.007077504166271598, -0.002450644438411309, 0.0013981132548902652, 0.018859354218815648, -0.021322918949281296, 0.01565923713984111, -0.025743098080254368, 0.020278560955135517, -0.005326067867368665, 0.024305510029027512, 0.16244744864546873, 0.1666942782512477, 0.20491854612347463, 0.15055611301818295, 0.1660871007470099, 0.1680095795435469, 0.18357773468836264, 0.16571190085897583, 0.18764763518921787, 0.15070400514890422, 0.16124219899212563, 0.24917934109097062, 0.16347687652388668, 0.20024063425969835, 0.17374135153923476, 0.18785053128520535, 0.20374027251188048, 0.20667248215324957, 0.2100408211314022, 0.157963837322002, 0.26892965140991665, 0.22166961540556884, 0.22959254345494023, 0.26213955617568685, 0.2519825241100336, 0.24718763494996865, 0.22839344210509274, 0.27186030437563674, 0.21334948099026146, 0.27107329897677457, 0.20577560655361027, 0.2148327864641996, 0.2361676896263822, 0.20567373741789274, 0.23542916799474045, 0.21259040713942343, 0.23695916160465078, 0.21801126215350955, 0.22643000500510657, 0.22062207470118594, -0.01659142616256352, 0.02535545567310787, 0.020432118377167543, -0.01475379768959964, 0.013049721401244503, 0.006348830072945637, 0.025005810372983446, 0.004845760696204816, -0.012874659828893532, 0.007669047739622425, 0.02074996783295549, -0.013763222339223226, -0.011351769254968674, 0.0055804093506285546, 0.02038191209519095, 0.014933085070520534, -0.01740367199879995, -0.025970950853632716, -0.0061586255622707855, 0.029941951264159234, -0.002142076497700873, -0.03371379958272194, -0.016961986727252125, 0.029084566656437116, 0.021292516601408035, -0.020569059972338963, 0.02419325246647849, -0.01197218478998678, 0.01919497936789196, 0.005471737860422745, 0.002506350650982768, -0.02098709140541721, -0.020117370422188616, 0.0061147526878901, -0.02750318006492234, 0.04381582799287355, -0.04465128194412741, -0.02503478475481979, 0.013420436127041043, -0.01552170740430413, 0.0021268093108138164, -0.001182850625360505, -0.008716226897995442, 0.00931732097370004, -0.0001519464253080975, -0.002612121492851625, 0.009974730207302124, 0.018994172045850643, 0.00036604888912976954, -0.00352315900005507, 0.0040155095489163945, 0.00915324314303018, 0.023479221640263254, 0.01437500880782188, -0.031483583389948194, 0.028838083148022334, -0.0030835667162377256, 0.0045409369014536545, 0.018676549198645215, 0.019215129974434822, 0.17185835069487426, 0.18642625329461382, 0.1526902211612833, 0.19857341456826821, 0.1529953473499201, 0.16573994390890118, 0.15064711551642276, 0.14612868242248145, 0.19443523593619494, 0.18367279132877767, 0.20790529375329592, 0.14304049941696773, 0.1992949183146291, 0.22797539421372792, 0.16283151214298475, 0.1715244104871768, 0.21399767013166998, 0.21396330161845514, 0.22226714758818905, 0.20005255174236192, 0.18919997230865887, 0.2411046875465966, 0.24201074743266782, 0.2223198250406924, 0.24859546537485677, 0.25941750332312635, 0.26667543306850355, 0.20627429538431227, 0.22505919043891284, 0.24185896477759022, 0.2091466605344864, 0.26812462025334893, 0.1753184175008922, 0.19740511589175483, 0.22460879996154676, 0.22407881137690372, 0.21673499191986753, 0.2494461672374472, 0.23596471699871618, 0.21766429019891487, 0.00023200970254691957, 0.01836803348136737, 0.0033750431922113366, 0.012236445904711065, 0.005606183175288101, 0.014034418038927821, 0.014615831528814871, 0.018258752519829242, 0.021783719470820567, 0.01572941927242213, -0.030489365807748286, 0.006686363935861093, 0.02403114084484552, 0.01244822649494708, 0.0014229340757363528, -0.04057830008765751, -0.0018887272036276256, -0.018905668107602826, 0.010222234165992694, -0.016177744477489993, -0.04406871942212976, 0.003973724846279114, 0.027370384520783914, -0.010221798373909737, -0.014442674866193116, -0.02021904480277859, -0.0045208627285134185, 0.04405651651502125, 0.03170245651029441, -0.043254695563397276, -0.03291613400567175, 0.04062573614710597, -0.020348542380411682, -0.03470720119457179, -0.02543696438147239, 0.0020881361513729745, 0.028430900423621765, -0.029826214988031122, -0.05283468769462476, -0.0032479129242025014, 0.021925457204342916, 0.028572997192538214, 3.819004787012289e-05, -0.052854661579761236, 0.005400466339044983, -0.003964164510107529, -0.0050167342459934055, 0.009404269013551906, 0.019606364976723877, 0.005292381918714394, 0.00944326702300299, -0.005368474665644606, 0.011855018436026826, 0.040888492235307855, -0.007891507811354302, -0.009891764609071704, 0.03331869562444887, 0.027273081315283543, 0.0042944783404620344, 0.03621266660788727, 0.15684503810375117, 0.17021742176893348, 0.1780979716738384, 0.15984199186766795, 0.13829559664629368, 0.17295629572468565, 0.17940908897979954, 0.16753046097300006, 0.15218068838049414, 0.19935220276183827, 0.1838685502051001, 0.15109184478476112, 0.18738903522521663, 0.21117583389159778, 0.16953816289024137, 0.2121362117848519, 0.22382219455896796, 0.1630726032308031, 0.19444902976144102, 0.21940275199854667, 0.23208765916033053, 0.2273134112769647, 0.2425296690367405, 0.21757714149788798, 0.21507734337544626, 0.24693105250550226, 0.238126293376356, 0.21283734195407172, 0.20641390493333692, 0.24456808544245742, 0.23138021279756787, 0.19987696635680133, 0.2204897807352658, 0.21268125248274788, 0.1777112773190452, 0.22947335894318005, 0.22070917029088627, 0.19865382529574582, 0.23169254810103682, 0.2211463490503829, 0.03751467926151041, -0.021473793687209575, 0.02070705093423588, 0.02379622720108372, -0.0021968551978916928, -0.004540602235516055, 0.03261618792651425, -0.013140829128071334, 0.02476805259973078, 0.03993260856168177, -0.035469158112519564, 0.02530726268610042, -0.011460674744034291, -0.07172529706805805, 0.005754650414018399, 0.03254323850603253, -0.03300788711270739, -0.011372370875895701, 0.03468498347986264, -0.007178320292963695, -0.011390084744017774, 0.00420534546304283, -0.023532051765852095, -0.02168604973363122, 0.00555041541556334, -0.0068015959458383405, 0.0042757316969279915, 0.012830770057982305, -0.007053389661811489, 0.019929280572923362, -0.01982997486136446, -0.03924195454820256, 0.0013087936362686076, -0.01994646871055571, -0.03635939641797755, -0.021490817671297652, -0.0052957696450478976, 0.01680636422892383, 0.0063825407221823555, -0.05796004304614821, -0.008018617039558596, -0.004414706820994392, -0.039749946496466036, 0.014477605149434058, -0.015730988264549512, -0.05364234044272572, -0.003956345027041447, -0.0005866060766890508, -0.027040982033792158, -0.002311628363513877, 0.03648227995044498, 0.005365283159139272, 0.018017797906047545, -0.011836284170046098, -0.007440165130475384, 0.03612000904802514, -0.030298660463920737, 0.005955368955937912, -0.002985691063105807, 0.03030806354789985, 0.17331671518131544, 0.1691325562102027, 0.14500698574161544, 0.1519739935143474, 0.16804337812358713, 0.18061652854769142, 0.1803386173327988, 0.17932927204363835, 0.17049219691732784, 0.19236919365265437, 0.21053218219404224, 0.181462452960863, 0.1768856900449483, 0.19115707809945595, 0.1819259085683467, 0.18119810106886958, 0.204983048295561, 0.21801364180886734, 0.21422165892568618, 0.19849532391318878, 0.20580130270828678, 0.24617862427314993, 0.2526907626958149, 0.2147534293032848, 0.18944604359323655, 0.21886548954510643, 0.256671437510052, 0.26646387825672163, 0.24003768756713312, 0.22084901493707576, 0.18134966140777767, 0.201360769721847, 0.2242664639819146, 0.17603000093558055, 0.2023524731731098, 0.20535884445859606, 0.18378054888557446, 0.19955709803779145, 0.2275924295290813, 0.18080573991559734, 0.01316004563611744, 0.0163234174083116, -0.006918921683700093, -0.011014248755155593, 0.01941767865364929, -0.017638460260677, 0.01355317141705814, -0.013832758403438429, -0.012784907381394482, 0.02145265913426121, -0.007614002883374294, -0.010022033001891256, -0.019060974049479824, -0.002543044965269346, -0.0026087259495750073, 0.003419659242639632, 0.028952014503720307, -0.013782927542885641, -0.019885617382678005, 0.03252621490774323, 0.031138357296396146, -0.021049736397194322, -0.027143578771833005, -0.0022534654850908434, -0.02723957237766774, 0.0010250306366127071, 0.02080366516142289, -0.030439629224218147, -0.009316781973647232, 0.03432403960113825, 0.0139872303816026, -0.018356703807452315, -0.005571724177548102, -0.01253126104064809, -0.041708678170966816, -0.005740945854048659, 0.023986078816470465, -0.005520529439543701, -0.014924055789315558, -0.009127266725212798, -0.012346385402500815, -0.03271258410097163, -0.027922755898361948, -0.012032623451483393, -0.03128691866618981, -0.022614975666100042, -0.03215990855186464, -0.014437240938008543, -0.01839464511574736, -0.050162014155579, -0.032349376370475774, 0.0373903305843471, -0.01895197886516537, -0.0071303279929306775, -0.009424290219213488, 0.009063210494055566, -0.0019017517086729692, -0.012815496349661492, -0.01264552552756819, 0.03904047369816027, 0.14466284426475987, 0.16338340896718415, 0.1657434972820371, 0.17388970774700907, 0.16100611052038202, 0.1746523421606368, 0.19625952626616244, 0.17336497597665307, 0.1644354562764823, 0.20473575593080665, 0.1860380286152975, 0.1846669077281865, 0.2007039658531077, 0.17040414912773025, 0.20626498170023244, 0.20709281182354325, 0.18540201100161555, 0.19925431420374273, 0.1704921624106103, 0.2154516321239021, 0.27598171475116107, 0.2418834314161124, 0.21825200575142362, 0.23878082020824024, 0.23074869218032895, 0.22009736781567374, 0.2510930634122867, 0.2511614555680457, 0.24613709212086987, 0.24414813142329458, 0.20043138310234365, 0.19196930394114345, 0.20423905845541973, 0.20138994929993564, 0.19890560034778307, 0.19339891417112495, 0.20692084671018454, 0.19775634332055705, 0.2200939316266307, 0.16054041433065813, -0.041112016465802884, 0.0010983452618076193, 0.01140089687935177, -0.022453924406188337, -0.005858306930704367, 0.007828491867241546, 0.008429087897875066, -0.018806121754429042, -0.03005678702745237, 0.030314261896269925, 0.010593656195507214, -0.03272022152463207, -0.007535488038784257, 0.011944117817208312, 0.001962870590445697, -0.0018991090118412335, -0.006115368431015338, 0.007106083141287004, 0.024473443712136243, -0.027713873742168837, -0.029387999539436428, 0.030419857982405093, 0.011180907211601263, -0.02452692827949331, -0.0035845193910539336, 0.012640779328581476, -0.015239812556032753, 0.002399973124365292, 0.013556528857648473, -0.01535022524537641, 0.016753864559653355, 0.015071199943077927, -0.02064143302805332, -0.0077367817542914954, -0.004461534686919323, -0.011632148894522857, -0.0021681771828420955, 0.009816973972521513, -0.005702315596066971, -0.00816313162158458, 0.00026866080617795945, -0.012017408185742746, -0.02323351253899391, -0.02537998307052247, 0.0071484836547461355, 0.00305582566981296, -0.038279822524737155, -0.004579291705262151, -0.001972621653025708, -0.04621330503896743]
chi2_1 = [0.0013719547398716849, -0.0025617975546624793, -0.0006837214513026252, 0.003759943622271502, -0.0022743360911721645, 0.0005533991912975002, -0.003936309944224801, 0.004940097812540405, -0.002196229601410328, 0.001424113077814718, 0.17679292310089545, 0.1739698372244268, 0.1666348779524843, 0.17618044469044686, 0.17635778759227866, 0.17155589320807815, 0.17162478501288997, 0.17983686855573902, 0.17355089215762892, 0.1733957507558772, 0.19436444345138493, 0.19622516462883344, 0.1987360272502819, 0.20135532714974683, 0.2008184805581326, 0.1990949865377297, 0.1962576932968134, 0.19656709750265244, 0.20369188470845812, 0.19894832420204084, 0.2397360200566707, 0.24224743369776294, 0.24453526132028022, 0.24620567109201666, 0.2429362859985464, 0.24777214082109472, 0.2397110089660471, 0.23311664306743599, 0.24921571249059457, 0.24832821320533027, 0.21797399980633017, 0.22327585215237883, 0.22396837951036697, 0.21782413387869598, 0.22003730852826023, 0.2193176554529987, 0.22000273424087494, 0.21594859358380264, 0.22409643638105042, 0.22482108428937167, 0.0041562791263013675, 0.002834864655807316, 0.0017888012960083782, 0.0008446279951414399, -0.0032559996855457936, 0.003520780318514922, -0.005052717383327111, 0.007533966593499599, 0.002475717410730722, 0.001223495946477095, -0.0018461751963993202, -0.003694779082901589, -0.009030170738949336, 0.0014912197161590767, 0.004119052881115863, -0.004044776111483063, 0.006116713326478804, 0.010112954447554471, -0.005026365930729967, 0.00010202104567970179, -0.0011291370310125793, -0.0036707826532017193, -0.004600113660588239, 0.003381579256547813, 0.004017629977371476, 0.006735178122908447, 0.008380175989186946, -0.0017853376828202102, -0.002217273741780499, 0.005977730343048904, 0.008993353795525119, -0.010688541907692311, -0.0011959834746223755, 0.014023496054847078, 0.004466689755237679, -0.007400159899194732, -0.01581121798182866, -0.00589409564516328, -0.0019072772133557636, -0.0010460200284803036, 0.0013424926384663185, 0.0071350744886219935, 0.006881845605904388, -0.006381492052205068, -0.00039771723601698846, 0.004904707070371447, -0.006481145836186238, -0.00661364333745922, 0.003982861768314581, 0.005138063366778843, 0.008181925712036013, 0.007693827489959197, -0.0010167831704175501, -0.004111050322091941, 0.006377532757376959, 0.010890863544604999, -0.009191646265918027, -0.0025002070100849146, 0.011009141778318111, 0.0050415793197849095, 0.1629969933155354, 0.1632099385478276, 0.1631888688062938, 0.1733854334567121, 0.17466962582548878, 0.17324819016617043, 0.17875589457022392, 0.17357443738029496, 0.16619087209879077, 0.17264802233061402, 0.20450036766285093, 0.18778795936061118, 0.1907554314411574, 0.20001944789777715, 0.20535992787286889, 0.20626655773921462, 0.19926466207113477, 0.19371759346045628, 0.1961991788818652, 0.2079007057441636, 0.24692737860670705, 0.2331092887302231, 0.24795730404318847, 0.2558236749084686, 0.24978497227142282, 0.23929834728535754, 0.22233197839969535, 0.23834864204902886, 0.2400258331935865, 0.24986684708550494, 0.222817726341846, 0.22361076849736114, 0.22482131979314854, 0.21659635454748125, 0.21411292576004967, 0.22608071009852065, 0.2160646528769343, 0.21742841023138548, 0.2187977821211926, 0.225164182282651, 0.004527616345926036, 0.0008694318208891498, -0.008496085450058893, 0.0014846666855040898, 0.01150476073585674, -0.006168944572956784, -0.004266468823772081, 0.003576526286703814, 0.006799300814776625, 0.005035329788245685, 0.0020093432196951256, -0.020055201890886526, -0.004986587106360946, -0.00431234814535284, 0.005063179252441886, 0.0039922436227892944, -0.0018039235503947942, 0.0030000766566558664, -0.00038861636282164046, -0.012474624635063354, 0.0028001564529055154, 0.0017440745006761626, -0.004236539964092661, -0.0029927936982405167, -0.0016665067532543882, 0.0054308059296884715, -0.002644182299728428, -0.004018378251091874, -0.0014801067996418281, 0.010358065152587, -0.0021535192693017107, -0.003571909116775965, -0.0053992336999395315, 0.01193074161195203, 0.0026204228002096614, -0.005926397469139746, -0.008234650820671078, -0.005936597553986353, 0.0009797735294452934, -0.00014043451844749624, 0.001904950262378497, 0.004106850983420999, -0.005507667648574489, 0.004923805551849469, -0.005872403532654326, -0.0009313848881526561, -0.001810117049574572, -0.002127443619311782, -0.004991472202349619, -0.0030429779917086454, -0.0035429675986356447, 0.0033331504803176667, -0.0025248825973561194, 0.002419886173239828, 0.00452370229523263, -0.0008131907431761587, -7.049904251612483e-05, -0.0036619909141958557, 0.003871249389002246, 0.0006541608881735085, 0.17367673493262664, 0.16467623813643903, 0.1693122775605966, 0.1695781507365339, 0.1755436398315945, 0.18218434142117432, 0.1644446910469653, 0.17392431487405755, 0.17859797029994126, 0.17333942928683949, 0.19647610658155648, 0.19755913199690012, 0.19299793450247094, 0.19972650944359074, 0.19104610666179864, 0.19655766749508463, 0.20046610739223752, 0.1982835795939252, 0.20378491638372478, 0.1987725482845658, 0.23916065793114197, 0.24382042804860227, 0.24140580599030254, 0.2519301814188654, 0.24265347738877102, 0.24218943785525607, 0.23563642394841974, 0.2481290983531882, 0.23556835156665076, 0.2472499236448385, 0.22424102533752016, 0.22057635932349579, 0.2206711058896526, 0.22715354522147538, 0.2182367373301615, 0.2192027845359762, 0.22093653901651417, 0.220651322697792, 0.22313771978458957, 0.21678899485792957]
chi2_2 = [0.0007728153433949148, -0.0005268151608919194, 0.0002753287470127692, 0.0017692532708088965, -0.0015748780858200278, 0.0011456998769452398, -0.0022745240963245525, 0.0038458606996353145, -0.00019119549929748675, -5.488573246332929e-05, 0.002603483482509098, -0.0002643484622390235, -0.004428998138993733, 0.0020222781190693323, 0.001487380264567084, -0.0014879675235336155, 0.0010955161738781338, 0.0037686617874138533, -0.0012665604556696289, 0.0006144156308108376, -0.002952378049266373, -0.0009407292563141157, 0.001019343929229835, 0.002713233723145016, 0.0008223230101656783, -0.00043165380515078686, -0.0001452253746650742, 0.0007940284396756637, 0.0019450140670222676, -0.00039938898328461, -0.003313605719235102, -0.003104131281922962, -8.617806505884143e-05, 0.002606710356794117, 0.0004133487972179314, 0.0019234092603666454, -0.003126972163168056, -0.007353845536901633, 0.006787356526176532, 0.001481264719806813, -0.0020533769801405286, 0.0032043700221761713, 0.0007320510416433273, -0.00551427240853034, 0.0008591513172006654, -0.002775067697622892, -0.0028336648391970067, -0.002127653447651971, 0.003120051188984243, -0.00016963950691303804, 0.0014979195533259162, 0.0020726346136953075, 0.004002660221586209, -0.0013168559901180763, -0.002410045532564, 0.0003058463696934732, -0.005749028882096786, 0.006714112852007907, 0.003819620621728789, 0.0019832141689786007, 0.1745567051332393, 0.16833275809190146, 0.16769862446503836, 0.1764030591138682, 0.17697482141078322, 0.17091952589086293, 0.1804023172767853, 0.17997040794655056, 0.1661519813546542, 0.17442472209350923, 0.19831389978214733, 0.19394722332334444, 0.1965989828702642, 0.19949207663219737, 0.19855293539206836, 0.2009115031442228, 0.20339240113098206, 0.1986444463064949, 0.19608535287029702, 0.1981992769049542, 0.2466452443825529, 0.23605426377246747, 0.24525549998268767, 0.25443578483649226, 0.24660402700084735, 0.24179058342308782, 0.23218744922192583, 0.2372557966338274, 0.2441968702013592, 0.24552827068381886, 0.22188175593828596, 0.2304304180338849, 0.22492826155586762, 0.2146336825048874, 0.22034686675983553, 0.2239675641536361, 0.21856016330716924, 0.21645325615279393, 0.22279875277081074, 0.22343826944204664, 0.0032729381840898887, 0.004152914066192759, 0.0002307542217260771, -0.004492633722496256, 0.001578929166024514, 0.003712386821598859, -0.006110271419087673, 0.0018770060342780052, 0.00902750193947812, 0.0026322921653186135, -0.00587963531040963, -0.008407278114725957, -0.007749980500881299, 0.001492542208086991, 0.0026841465792893215, -0.0006862457991035442, 0.005360872746974398, 0.0011593608765038896, -0.0012841950418253031, -0.004229795030312734, 0.0021339163291882817, -0.006572013221165178, -0.0036430850538725756, -6.740583937321794e-05, 0.0028141275936983067, 0.00408848305111796, -2.366574424295387e-05, -0.002930625911795593, -0.0013189386749757847, 0.0030075370642672263, 0.0036985393763404244, -0.00779510334321888, 0.006015643836951509, 0.010416319627299647, 0.0037555573254611802, -0.0011356227596895968, -0.019369333851107902, -0.0014792216948569465, -0.005859827377464981, 0.004840454868104629, 0.0017604718821276014, 0.004326239440173086, 0.0034604358430867046, -0.0023900816759878326, -0.0038651971169762394, 0.00044328859955787493, -0.003932194514755824, -0.0014680351821868252, -0.003750770784114808, 0.0024591445341655247, 0.001461205722909773, -0.0013239767136760225, -0.004967479173896997, 0.00040376749596391276, 0.005892981480142891, -0.0011717253951026767, -0.0014569800249302932, 0.003073200495074628, 0.006976659322454869, 0.002260592622584344, 0.17503253227585489, 0.157253259706739, 0.17169128512948442, 0.16961205744609575, 0.1763381038414999, 0.17486385951555516, 0.1749436354871704, 0.17381059330674686, 0.1765326646565545, 0.16687421280219192, 0.19826262609479567, 0.19856070715442162, 0.19579233473874233, 0.19519964129344983, 0.19577034849951025, 0.20028890251482756, 0.19359010797289902, 0.1972207510450249, 0.19660631594645403, 0.20415042802384448, 0.24458533089457485, 0.24315886509376616, 0.24136171441764298, 0.25112553720963277, 0.24426402005879258, 0.24021304558562773, 0.23638565016523905, 0.24367529605649288, 0.2435781903740091, 0.24115231487278332, 0.22363966299380827, 0.22270511953906044, 0.21747005424262902, 0.22585509628549866, 0.2193880413564922, 0.2214940826479367, 0.22095690955624334, 0.21991178965293706, 0.21676519928978272, 0.21894822690787985, -0.0004824009764538746, -0.0005882670726823013, 0.00014870205964649585, 0.0031514748269758706, 0.0017987038465356946, -3.83606907504116e-06, 0.00010219356278415787, 0.000845457694418783, -0.00032703421653724877, -0.0010843653220578777, 0.0018612233557527879, -0.006359040797858324, -0.00037908089845776127, -0.0036490148692434944, 0.0025748139844341696, 0.006725165269831427, -0.007496678736017416, 0.0010130153074746978, 0.002546012254506252, -0.0002583272811766944, -0.0013206785464016835, 0.001297445415739386, -0.0018311215184720867, -0.0023418037150217193, -0.002793709664031025, -2.9330767371355063e-05, -0.00010239436547141939, -0.00073781300171776, 0.0049599069932587435, 0.0019299935759001731, -0.0032044394603184872, 0.00212822700944382, -0.0044847944136237545, 0.005841773843414869, -0.002577422647434414, -0.0009216323915395274, -0.0029950716342855167, 0.0020916479809743984, -0.003109609870412765, 0.002041971947332536, 0.0018471030627003784, -0.0009536440391868873, -0.0012630175467611131, 0.0030591732459336763, -0.0019803123502033725, -0.001896130362299988, 0.004576478124624437, -0.0023380818860989482, 0.0008813132681075796, -0.00221584165465073]

chi3 = []
chi3.extend(chi2_1)
chi3.extend(chi2_2)

print(len(chi1))
print(len(chi3))
print(np.allclose(chi1,chi3,2))